In [1]:
import pandas as pd
import geopandas as gpd
import os
from os import walk
import zipfile
import re
import numpy as np
from itertools import chain
import time
import collections
import datetime

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
inpath = r'L:\NaturalResources\Wetlands\Local Wetland Inventory\WAPO\EPA_2022_Tasks\Task 1 WD Mapping'

In [3]:
wdpath = inpath + '\\DSL data originals\Set001'

In [4]:
txpath = inpath + '\\GIS\ORMAP_data\Downloaded zip files'

In [5]:
newpath = os.path.join(inpath, "test")

In [8]:
# create a temporary folder to run a test - one-time running, can remove the folder to rerun
# os.mkdir(newpath)

In [88]:
def list_files(path, folder=False):
    f = []
    for (dirpath, dirnames, filenames) in walk(path):
        f.extend(filenames)
        break
    if folder:
        f = [x[0] for x in os.walk(path)]
    return f

In [14]:
list_files(wdpath)

['ClatsopCoDelsDets2017-20220601.xlsx',
 'CoosCo2017-20220601.xlsx',
 'DeschutesCoDelDet2017-20220601.xlsx',
 'HarneyCo2017-20220601.xlsx',
 'JacksonCo2017-20220601.xlsx',
 'JosephineCo2017-20220601.xlsx',
 'LakeCoDelDet2017-20220601.xlsx',
 'MarionCoDelDet2017-20220601.xlsx',
 '~$ClatsopCoDelsDets2017-20220601.xlsx',
 '~$CoosCo2017-20220601.xlsx']

In [15]:
list_files(txpath)

['2012 Taxlots.zip',
 '2014_state_agency_use_taxlots.gdb.zip',
 '2015_state_agency_use_taxlots.gdb.zip',
 '2016ShareableTaxLotsGDBJuneUpdate.zip',
 'Morrow2015.zip',
 'Shared_Taxlots_2017_GDB.zip',
 'Shared_Taxlots_2018_GDB.zip',
 'Shared_Taxlots_2019_GDB.zip',
 'Shared_Taxlots_2020_GDB.zip',
 'Shared_Taxlots_2021_GDB.zip',
 'Shared_Taxlots_2022_GDB.zip',
 'State Data2009.gdb.zip',
 'taxlots_2011.zip']

In [85]:
year = 2017

In [86]:
# one-time runnning
with zipfile.ZipFile(os.path.join(txpath, 'Shared_Taxlots_'+str(year)+'_GDB.zip'), 'r') as zip_ref:
    zip_ref.extractall(newpath)

In [90]:
# how the original taxlot data is stored
list_files(newpath, folder=True)

['L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\test',
 'L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\test\\StateAgencyTaxlots2017.gdb',
 'L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\test\\Taxlots2022',
 'L:\\NaturalResources\\Wetlands\\Local Wetland Inventory\\WAPO\\EPA_2022_Tasks\\Task 1 WD Mapping\\test\\Taxlots2022\\Shared Taxlots.gdb']

In [28]:
# rename the folder, one-time runnning
os.renames(list_files(newpath, folder=True)[1], newpath+'\\Taxlots'+str(year))

In [142]:
datafile = os.path.join(wdpath, 'DeschutesCoDelDet2017-20220601.xlsx')

In [143]:
# check the sheets in the wd spreadsheets
xl = pd.ExcelFile(datafile)
xl.sheet_names

['Delineations-Determinations', 'Reorg']

In [144]:
# read the reorganized table in the raw data
wd_dt = pd.read_excel(datafile, sheet_name='Reorg')

In [145]:
wd_dt.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id'],
      dtype='object')

In [33]:
wd_dt.head(3)

,wetdet_delin_number,trsqq,parcel_id,address_location_desc,city,county,site_name,site_desc,latitude,longitude,Coord-Source,DocumentName,DecisionLink,is_batch_file,status_name,received_date,response_date,reissuance_response_date,project_id,site_id
0,WD2017-0003,06N10W03DA,500 (partial),McCormick Rd .025 mi S of Hillila Rd,Gearhart,Clatsop,McCormick Rd 06N10W03DA,Residential,46.030197,-123.905217,DSL Verified,WD2017-0003final,https://docs.dsl.state.or.us/PublicReview/docv...,N,Approved,2017-01-03,2017-03-09,NaN,NaN,103777
1,WD2017-0033,08N08W25,101,"W of 92260 Koppisch Rd, Astoria",Astoria,Clatsop,08N08W25,site evaluation,46.155778,-123.610751,DSL Verified,WD20170033AgencyDecision,https://docs.dsl.state.or.us/PublicReview/Elec...,Y,Completed by DSL,2017-01-12,2017-01-30,NaN,NaN,104259
2,WD2017-0056,08N10W09B,"2500, ROWs (both partial)",Warrenton Dr and NW 17th St,Warrenton,Clatsop,Gann Dev 08N10W09B,residential development,46.190638,-123.932177,DSL Verified,WD2017-0056final,https://docs.dsl.state.or.us/PublicReview/docv...,N,Approved,2017-02-06,2017-05-17,NaN,NaN,104411


In [146]:
wd_dt.parcel_id.dtype

dtype('O')

In [96]:
#txfilepath = os.path.join(newpath, 'Taxlots'+str(year), 'Shared Taxlots.gdb')
#txfilepath = os.path.join(newpath, 'StateAgencyTaxlots' + str(year) + '.gdb')
txfilepath = os.path.join(inpath, 'GIS', 'ORMAP_data', 'ORMAP_Taxlot_Years', 'Taxlots2017.gdb')

In [100]:
# this will take a while
start = time.time()
tx_dt = gpd.read_file(txfilepath, layer='TL_Dissolv')
end = time.time()
print(end - start)

181.2430136203766


In [42]:
# just check a smaller set
tx_dt_s = gpd.read_file(txfilepath, layer='Taxlots', rows=10)

In [102]:
tx_dt.columns

Index(['ORTaxlot', 'Shape_Length', 'Shape_Area', 'geometry'], dtype='object')

In [150]:
tx_dt.crs

<Projected CRS: EPSG:2992>
Name: NAD83 / Oregon GIC Lambert (ft)
Axis Info [cartesian]:
- X[east]: Easting (foot)
- Y[north]: Northing (foot)
Area of Use:
- name: United States (USA) - Oregon.
- bounds: (-124.6, 41.98, -116.47, 46.26)
Coordinate Operation:
- name: Oregon GIC Lambert (International feet)
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [46]:
# check the format of ORTaxlot
tx_dt_s.ORTaxlot.head(3)

0    0106.00S38.00E0000--000000300
1    0109.00S40.00E2800--000000800
2    0106.00S38.00E0000--0000ROADS
Name: ORTaxlot, dtype: object

In [13]:
# create a spreadsheet to create a dictionary
cnt_ID = pd.read_excel(r'T:\DCProjects\EPA-WD\CNT_Code.xlsx')

In [14]:
# create a dictionary to look up county code
cnt_dict = dict(zip(cnt_ID.COUNTY, cnt_ID.ID))

In [148]:
# check how parcel id is noted
wd_dt.parcel_id.unique()

array(['600', '100,102,301,501,502,503', '302', '2500', '505', '4600',
       '2900', '100', '99', '2100', '1500', '800 (portion)', '104',
       '300 portion', '700 portion', '99 partial',
       '1300,1400 both partial', '101', '1400', '5700', '403', '406',
       '2200', '200 partial', '104 partial', '1100 partial',
       '316,313,309,303,101 all partial', '4700, 5000 (portion)',
       '99, 14000 (portion)', '7100, 7200 (portion)',
       'Newport Ave ROW partial, 4300,4400', '1500,1600', '4000,4001',
       '4500,4600,4700,5300,5301,5400,', '4400,4500', '5400,7100', '4700',
       '300', '809,817,829', '5100', '4900', '610,615,619', nan,
       '1300,1400,1500,1501,1700,1702,2000', '1102,1103',
       '102,2400,2500,2600', '1300', '2600,2700,2071', '1800,1900,2200',
       '2200,2300,2400,2500,2600,2700,2800,2900',
       '100,200,300,400,500,600,700,800', '2800', '200', '800,803',
       '2101', '600,700', '4500', '1403', '400,500,600,700',
       '100,200 all partial', '11200',

In [69]:
# this id looks different
'100-10600 (106 TLs)'.replace('(','').replace(')','')  

'100-10600 106 TLs'

In [15]:
# function to remove duplicates in the parcel ids
def unique(list1):
    x = np.array(list1)
    return list(np.unique(x))

In [16]:
# function to get all the lot numbers
def get_lot_numbers(text):
    # remove parenthesis from text
    txt = text.replace('(','').replace(')','')
    # split the text
    lot_list = re.split(",|, | |-", txt)
    # remove text elements
    l = []
    # in case there are still number-letter strings (e.g., '1a')
    for t in [lot for lot in lot_list if ~lot.isnumeric()]:
        if any(c.isdigit() for c in t):
            l.append(re.sub('\D', '', t))
    res = unique([lot for lot in lot_list if lot.isnumeric()] + l)
    return res

In [115]:
# use a random data subset for a test run
#tdf = wd_dt.sample(n=3, random_state=1)

In [149]:
# get a list of lot numbers in each parcel id record
#tdf.loc[:, 'lots'] = tdf.parcel_id.apply(lambda x: get_lot_numbers(x))
wd_dt = wd_dt[wd_dt.parcel_id.astype(str) != 'nan']
wd_dt.loc[:, 'lots'] = wd_dt.parcel_id.apply(lambda x: get_lot_numbers(x))

In [18]:
# repeat the rows based on the number of lot numbers
#ndf = tdf.reindex(tdf.index.repeat(tdf.lots.str.len()))
ndf = wd_dt.reindex(wd_dt.index.repeat(wd_dt.lots.str.len()))

In [20]:
# add the column to list the lot for all
#ndf.loc[:, 'lot'] = list(chain.from_iterable(tdf.lots.values.tolist()))
ndf.loc[:, 'lot'] = list(chain.from_iterable(wd_dt.lots.values.tolist()))

In [21]:
# get county code
ndf.loc[:, 'cnt_code'] = ndf.county.map(cnt_dict)

In [22]:
# get OR taxlot IDs for wd data
ndf.loc[:, 'ORTaxlot'] = ndf[['cnt_code', 'trsqq', 'lot']].apply(lambda row: str(row.cnt_code).zfill(2) + row.trsqq[0:2] + '.00' + row.trsqq[2:5] + '.00'+ row.trsqq[5:8] + '{:0<10}'.format(row.trsqq)[8] + '{:0<10}'.format(row.trsqq)[9] + '--' + ('000000000' + row.lot)[-9:], axis = 1) 

In [147]:
ndf.trsqq.unique()

array(['08N10W26', '07N10W16', '08N10W22DA'], dtype=object)

In [160]:
#0106.00S38.00E0000--000000300
ndf.ORTaxlot.unique()

array(['0408.00N10.00W2600--000000500', '0407.00N10.00W1600--000000200',
       '0407.00N10.00W1600--000000300', '0408.00N10.00W22DA--000001001',
       '0408.00N10.00W22DA--000001002', '0408.00N10.00W22DA--000001100'],
      dtype=object)

In [155]:
txlist = list(tx_dt.ORTaxlot.unique())

In [162]:
# check if the taxlot ID is in the correct format
all([x in txlist for x in ndf.ORTaxlot.unique()])

True

In [163]:
# get a taxlot data subset for a join test run
tx_dt_s = tx_dt[tx_dt.ORTaxlot.isin(ndf.ORTaxlot.unique())]

In [23]:
# there is not any repeated taxlot ID in this taxlot feature subset
#len(tx_dt_s.ORTaxlot.unique()) == tx_dt_s.shape[0]
# there are repeated taxlot ID in the taxlot feature
len(tx_dt.ORTaxlot.unique()) == tx_dt.shape[0]

False

In [24]:
# there is not any repeated taxlot ID in this wd data subset
len(ndf.ORTaxlot.unique()) == ndf.shape[0]
# there are repeated taxlot IDs in the wd data

False

In [167]:
# in this case it is a one-on-one join
merged = tx_dt_s[['MapTaxlot', 'ORTaxlot', 'geometry']].merge(ndf[['wetdet_delin_number','address_location_desc', 'ORTaxlot']], 
                                                     on='ORTaxlot')

In [171]:
merged

,MapTaxlot,ORTaxlot,geometry,wetdet_delin_number,address_location_desc
0,710160000200,0407.00N10.00W1600--000000200,"MULTIPOLYGON (((443805.689 1602370.314, 443880...",WD2020-0280,W of 90266 Hwy 101
1,710160000300,0407.00N10.00W1600--000000300,"MULTIPOLYGON (((444458.870 1600237.427, 444539...",WD2020-0280,W of 90266 Hwy 101
2,81022DA01001,0408.00N10.00W22DA--000001001,"MULTIPOLYGON (((449719.082 1626698.536, 449715...",WD2020-0402,SE 2nd and E Harbor
3,81022DA01002,0408.00N10.00W22DA--000001002,"MULTIPOLYGON (((449769.062 1626696.689, 449819...",WD2020-0402,SE 2nd and E Harbor
4,81022DA01100,0408.00N10.00W22DA--000001100,"MULTIPOLYGON (((449669.104 1626700.383, 449665...",WD2020-0402,SE 2nd and E Harbor
5,810260000500,0408.00N10.00W2600--000000500,"MULTIPOLYGON (((452569.653 1623833.664, 452590...",WD2020-0468,(no discrete address) SE 12th Pl


In [33]:
def check_duplicates(v):
    itemlist = [item for item, count in collections.Counter(v).items() if count > 1]
    return itemlist, len(itemlist)

In [35]:
# check the repeated IDs
check_duplicates(ndf.ORTaxlot.values)[1]

27

In [31]:
type(tx_dt.County.unique()[4])

numpy.int64

In [30]:
ndf.cnt_code.unique()

array([4], dtype=int64)

In [103]:
#check_duplicates(tx_dt[tx_dt.County == 4].ORTaxlot.values)[1]
check_duplicates(tx_dt.ORTaxlot.values)[1]

0

In [38]:
all([v in check_duplicates(tx_dt[tx_dt.County == 4].ORTaxlot.values)[0] for v in check_duplicates(ndf.ORTaxlot.values)[0]]) 

False

In [41]:
[v for v in check_duplicates(ndf.ORTaxlot.values)[0] if v in check_duplicates(tx_dt[tx_dt.County == 4].ORTaxlot.values)[0]]

['0408.00N09.00W14DD--000000100']

In [44]:
ndf[ndf.ORTaxlot == '0408.00N09.00W14DD--000000100'].received_date

55   2019-05-06
57   2019-06-11
Name: received_date, dtype: datetime64[ns]

In [51]:
ndf.loc[:, 'year'] = ndf.received_date.apply(lambda x: x.year)

In [136]:
ndf.loc[:, 'IDyear'] = ndf.wetdet_delin_number.apply(lambda x: x[2:6])

In [55]:
check_duplicates(ndf[ndf.year == year].ORTaxlot.values)[1]

0

In [95]:
# # this will take a while
# start = time.time()
# dissoved = tx_dt.dissolve('ORTaxlot')
# end = time.time()
# print(end - start)

In [61]:
check_duplicates(dissoved.index)[1]

0

In [62]:
dissoved.loc[:, 'ORTaxlot'] = dissoved.index

In [104]:
tx_dt.columns

Index(['ORTaxlot', 'Shape_Length', 'Shape_Area', 'geometry'], dtype='object')

In [65]:
dissoved.reset_index(drop=True, inplace=True)

In [137]:
merged = ndf[ndf.IDyear == str(year)].merge(tx_dt[['ORTaxlot', 'geometry']], on='ORTaxlot', how='left')

In [106]:
merged.head(3)

,wetdet_delin_number,trsqq,parcel_id,address_location_desc,city,county,site_name,site_desc,latitude,longitude,...,response_date,reissuance_response_date,project_id,site_id,lots,lot,cnt_code,ORTaxlot,year,geometry
0,WD2017-0003,06N10W03DA,500 (partial),McCormick Rd .025 mi S of Hillila Rd,Gearhart,Clatsop,McCormick Rd 06N10W03DA,Residential,46.030197,-123.905217,...,2017-03-09,NaN,NaN,103777,[500],500,4,0406.00N10.00W03DA--000000500,2017,"MULTIPOLYGON (((448821.559 1578307.121, 448814..."
1,WD2017-0033,08N08W25,101,"W of 92260 Koppisch Rd, Astoria",Astoria,Clatsop,08N08W25,site evaluation,46.155778,-123.610751,...,2017-01-30,NaN,NaN,104259,[101],101,4,0408.00N08.00W2500--000000101,2017,"MULTIPOLYGON (((524028.003 1621190.023, 524220..."
2,WD2017-0056,08N10W09B,"2500, ROWs (both partial)",Warrenton Dr and NW 17th St,Warrenton,Clatsop,Gann Dev 08N10W09B,residential development,46.190638,-123.932177,...,2017-05-17,NaN,NaN,104411,[2500],2500,4,0408.00N10.00W09B0--000002500,2017,"MULTIPOLYGON (((443198.754 1636916.627, 443123..."


In [71]:
set1gdf = gpd.read_file(os.path.join(inpath, 'GIS', 'Join_Statewide.gdb'), layer='WD_Set001_Combined')

In [138]:
merged.shape

(33, 27)

In [76]:
set1gdf.columns

Index(['Record_ID', 'wetdet_delin_number', 'trsqq', 'parcel_id',
       'address_location_desc', 'city', 'county', 'site_name', 'site_desc',
       'latitude', 'longitude', 'Coord_Source', 'DocumentName', 'DecisionLink',
       'is_batch_file', 'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'lot_num', 'lot',
       'Match_Year', 'ORTAXLOT', 'Match_found', 'Manual_note', 'Shape_Length',
       'Shape_Area', 'Edits_Complete', 'matcher2', 'geometry'],
      dtype='object')

In [82]:
set1gdf.loc[:, 'year'] = set1gdf.received_date.values[0].split('-')[0]

In [120]:
set1gdf.loc[:, 'IDyear'] = set1gdf.wetdet_delin_number.apply(lambda x: x[2:6])

In [83]:
set1gdf.year.unique()

array(['2017'], dtype=object)

In [110]:
set1gdf[set1gdf.county == 'Clatsop'].shape

(241, 33)

In [121]:
set1gdf[set1gdf.IDyear == '2017'].shape

(216, 34)

In [128]:
set1gdf[(set1gdf.county == 'Clatsop') & (set1gdf.IDyear == '2017')].shape

(33, 34)

In [111]:
check_duplicates(merged.ORTaxlot.values)[1]

2

In [113]:
set1gdf[set1gdf.wetdet_delin_number.isin(merged.wetdet_delin_number.unique())].shape

(35, 33)

In [84]:
set1gdf.head(3)

,Record_ID,wetdet_delin_number,trsqq,parcel_id,address_location_desc,city,county,site_name,site_desc,latitude,...,Match_Year,ORTAXLOT,Match_found,Manual_note,Shape_Length,Shape_Area,Edits_Complete,matcher2,geometry,year
0,1188.0,WD2017-0552,06S02W09,"800, 1800, 1900","6296 Umpqua St NE, Salem",Salem,Marion,Cottontail Solar LLC 06S02W09,Request for a jurisdictional determination for...,45.067881,...,2017.0,None,1.0,None,6291.224348,1.237596e+06,NaN,1188lot2,"MULTIPOLYGON (((682695.722 1217565.770, 683060...",2017
1,978.0,WD2017-0241,37S05W18,200,"827 Triller Ln, Grants Pass",Grants Pass,Josephine,37S05W18B,Faye Benson Trust,42.357634,...,2017.0,1737.00S05.00W1800--000000200,1.0,None,5550.903822,1.913187e+06,NaN,978lot1,"MULTIPOLYGON (((544163.432 236069.011, 544115....",2017
2,614.0,WD2017-0407,36S04W16CD,"500,502",Sunna Way/Foothill Blvd/Westbook Dr,Rogue River,Jackson,36S04W16CD,Assessment of common areas,42.434712,...,2017.0,None,1.0,None,900.078840,1.929808e+04,NaN,614lot2,"MULTIPOLYGON (((588847.611 261808.947, 588803....",2017


In [118]:
set(list(map(lambda x:x[2:6], 
         set1gdf[~set1gdf.wetdet_delin_number.isin(merged.wetdet_delin_number.unique())].wetdet_delin_number.values)))

{'2017', '2018', '2019', '2020', '2021', '2022'}

In [124]:
missing_list = set1gdf[(~set1gdf.wetdet_delin_number.isin(merged.wetdet_delin_number.unique())) & (set1gdf.IDyear == '2017')].wetdet_delin_number.unique()

In [126]:
len(missing_list)

119

In [125]:
[ID for ID in missing_list if ID in ndf]

[]

In [127]:
set1gdf[set1gdf.wetdet_delin_number.isin(missing_list)].county.unique()

array(['Marion', 'Josephine', 'Jackson', 'Coos', 'Deschutes', 'Lake',
       'Harney'], dtype=object)

In [130]:
setIDs = set1gdf[(set1gdf.county == 'Clatsop') & (set1gdf.IDyear == '2017')].wetdet_delin_number.unique()

In [133]:
merged[~merged.wetdet_delin_number.isin(setIDs)].wetdet_delin_number.unique()

array(['WD2018-0036'], dtype=object)

In [ ]:
'0407.00N10.00W16AA--000000100'

In [141]:
60*60*3/0.1

108000.0